# Imports

In [ ]:
import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install scikit-learn
!{sys.executable} -m pip install pillow

In [ ]:
import math
import subprocess
import tkinter as tk
import turtle

import numpy as np
from scipy.stats import linregress
from PIL import Image

# Draw image

In [ ]:
def dump_gui(canvas, filename = "gui_image_grabbed.png"):
    """
    takes a png screenshot of a tkinter window, and saves it on in cwd
    """
    print(f'...dumping gui window to png: {filename}')
    import io
    
    ps = canvas.postscript(colormode="mono")
    im = Image.open(io.BytesIO(ps.encode('utf-8')))
    im.save(filename)

## Sierpinski Triangle

In [ ]:
def draw_sierpinski(turtle, length, depth):
    if depth == 0:
        for i in range(0, 3):
            turtle.fd(length)
            turtle.left(120)
    else:
        draw_sierpinski(turtle, length / 2, depth - 1)
        turtle.fd(length / 2)
        draw_sierpinski(turtle, length / 2, depth - 1)
        turtle.bk(length / 2)
        turtle.left(60)
        turtle.fd(length / 2)
        turtle.right(60)
        draw_sierpinski(turtle, length / 2, depth - 1)
        turtle.left(60)
        turtle.bk(length / 2)
        turtle.right(60)

### Run

In [ ]:
root = tk.Tk()
canvas = tk.Canvas(root, width=500, height=500)
canvas.pack()

t = turtle.RawTurtle(canvas)

t.penup()
t.goto(-200, -175)
t.pendown()
draw_sierpinski(t, 400, 2)
t.hideturtle()

dump_gui(canvas, "Triangle1.png")

root.mainloop()

## Julia Sets

In [ ]:
def draw_julia(cX = -0.7, cY = 0.25, moveX = 0.0, moveY = 0.0, maxIter = 255):
    # setting the width, height and zoom 
    # of the image to be created
    w, h, zoom = 800,500,0.75

    # creating the new image in RGB mode
    bitmap = Image.new("L", (w, h), "white")

    # Allocating the storage for the image and
    # loading the pixel data.
    pix = bitmap.load()

    for x in range(w):
        for y in range(h):
            zx = 1.5*(x - w/2)/(0.5*zoom*w) + moveX
            zy = 1.0*(y - h/2)/(0.5*zoom*h) + moveY
            i = maxIter
            while zx*zx + zy*zy < 4 and i > 1:
                tmp = zx*zx - zy*zy + cX
                zy,zx = 2.0*zx*zy + cY, tmp
                i -= 1

            # convert byte to RGB (3 bytes), kinda 
            # magic to get nice colors
            # pix[x,y] = (i << 21) + (i << 10) + i*8
            pix[x,y] = i*8

    # to display the created fractal
    return bitmap

### Run

In [ ]:
julia = draw_julia(-1, 0)

julia.show()

In [ ]:
julia.save("Julia -1.png")

## Koch Curve

In [ ]:
#function to create koch snowflake or koch curve 
def snowflake(turtle, lengthSide, levels): 
    if levels == 0: 
        turtle.forward(lengthSide) 
        return
    lengthSide /= 3.0
    snowflake(t, lengthSide, levels-1) 
    turtle.left(60) 
    snowflake(t, lengthSide, levels-1) 
    turtle.right(120) 
    snowflake(t, lengthSide, levels-1) 
    turtle.left(60) 
    snowflake(t, lengthSide, levels-1) 

### Run

In [ ]:
root = tk.Tk()
canvas = tk.Canvas(root, width=1920, height=1080)
canvas.pack()

t = turtle.RawTurtle(canvas)

t.speed(0)

t.penup()
length = 1800.0              
#t.goto(-200, -175)
t.backward(length/2.0)
t.pendown()
snowflake(t, length, 6) 
t.hideturtle()

dump_gui(canvas, "Koch Curve.png")

root.mainloop()  

In [ ]:
part_ratio = 2 * math.cos(math.radians(72))
side_ratio = 1 / (part_ratio + 2)

hide_turtles = True   # show/hide turtles as they draw
path_color = "black"  # path color
fill_color = "black"  # fill color

# turtle, size
def pentagon(t, s):
  t.pendown()
  t.right(36)
  t.begin_fill()
  for i in range(5):
    t.forward(s)
    t.right(72)
  t.end_fill()

# iteration, turtle, size
def sierpinski(canvas, i, t, s):
  t.setheading(0)
  new_size = s * side_ratio
  
  if i > 1:
    i -= 1
    
    # create four more turtles
    for j in range(4):
      t.right(36)
      short = s * side_ratio / part_ratio
      dist = [short, s, s, short][j]
      
      # spawn a turtle
      spawn = turtle.RawTurtle(canvas)
      spawn.speed(0)
      if hide_turtles:spawn.hideturtle()
      spawn.penup()
      spawn.setposition(t.position())
      spawn.setheading(t.heading())
      spawn.forward(dist)
      
      # recurse for spawned turtles
      sierpinski(canvas, i, spawn, new_size)
    
    # recurse for parent turtle
    sierpinski(canvas, i, t, new_size)
    
  else:
    # draw a pentagon
    pentagon(t, s)
    # delete turtle
    del t

In [ ]:
root = tk.Tk()
canvas = tk.Canvas(root, width=1920, height=1080)
canvas.pack()

t = turtle.RawTurtle(canvas)

t.speed(0)
t.penup()
t.hideturtle()
screen = t.getscreen()
y = screen.window_height()
t.goto(0, y/2-20)

i = 5       # depth. i >= 1
size = 600  # side length

# so the spawned turtles move only the distance to an inner pentagon
size *= part_ratio

# begin recursion
sierpinski(canvas, i, t, size)

dump_gui(root, "Pentagon.png")

root.mainloop()

# Run C++ executable

In [ ]:
runTimes = 10
filename = "Pentagon.png"

subprocess.call(args=["./build/FractalDimensionComputation", "-f", f"{filename}", "-t", f"{runTimes}"], stdin=None, stdout=None, stderr=None, shell=False)

# Calculate statistics

In [ ]:
runTimes = 10
times = []
with open("results_time.txt", "r") as fin:
    for _ in range(runTimes):
        times.append(float(fin.readline()[:-1]))

for i in range(runTimes):
    sizes = [] # x = s
    boxes = [] # y = n(s)
    with open(f"results_{i}.txt", "r") as fin:
        line = fin.readline()[:-1].split(' ')
        while line != [""]:
            sizes.append(np.log(1 / int(line[0])))
            boxes.append(np.log(int(line[1])))
            
            line = fin.readline()[:-1].split(' ')
    
    print("----------------------------------")
    print(f"Run {i}")
    print(f"Dimension = {linregress(sizes, boxes).slope}")
    print(f"Time spent: {times[i]}")
print(f"Time spent mean: {np.mean(times)}")
print(f"Time spent standard deviation: {np.std(times)}")